In [7]:
from nn_generalisation import neural_net
import torch
from torch.utils.data import DataLoader

SyntaxError: invalid syntax (1515834065.py, line 1)

In [ ]:
def train(model : neural_net.DenseNN,
          train_loader : DataLoader,
          optimizer : torch.optim.SGD,
          loss_fn : torch.nn.modules.loss._Loss | torch.nn.modules.loss._WeightedLoss,
          args : dict) -> float:
    model.train()
    epoch_loss = 0
    for batch_idx, (data, target) in enumerate(train_loader):
        if not args["pre_transfer"]:
            data, target = data.to(args["device"]), target.to(args["device"])
        optimizer.zero_grad()
        output = model(data)
        loss = loss_fn(output, target) / len(data)
        epoch_loss += loss * len(data)
        loss.backward()
        grads = torch.Tensor([-optimizer.lr * p.grad for p in model.parameters()], device=args["device"])
        optimizer.step()
    return epoch_loss.item() / len(train_loader.dataset), grads

def get_landscape_metrics(model : neural_net.DenseNN,
        test_loader : DataLoader,
        loss_fn : torch.nn.modules.loss._Loss | torch.nn.modules.loss._WeightedLoss,
        args : dict,
        v : torch.Tensor) -> float:
    model.eval()
    test_loss = 0
    for data, target in test_loader:
        if not args["pre_transfer"]:
            data, target = data.to(args["device"]), target.to(args["device"])
        output = model(data)
        test_loss += loss_fn(output, target) / len(data)
    test_loss.backward(create_graph=True)
    grad_params = torch.cat([p.grad.view(-1) for p in model.parameters()])
    hv_product = torch.autograd.grad(grad_params, model.parameters(), grad_outputs=v, retain_graph=True)
    return test_loss.item()